In [31]:
import pandas as pd
import shap
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, r2_score, mean_absolute_error, mean_squared_error
import ast
import numpy as np

In [2]:
df = pd.read_csv('../../feature_eng/example_features.csv')
df = df.drop(['Unnamed: 0', 'id'], axis=1)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['score'], axis=1), df['score'], test_size=0.10, stratify=df['score'], random_state=42)

In [4]:
def get_shap_values(model, x_train):

    assert type(x_train)==pd.core.frame.DataFrame

    shap_explainer = shap.TreeExplainer(model)
    shaps = pd.DataFrame(
        data=shap_explainer.shap_values(x_train),
        index=x_train.index,
        columns=x_train.columns
    )

    return shaps

In [5]:
def get_feature_contributions(y_true, y_pred, shap_values):
  """Compute prediction contribution and error contribution for each feature."""

  prediction_contribution = shap_values.abs().mean().rename("prediction_contribution")

  abs_error = (y_true - y_pred).abs()
  y_pred_wo_feature = shap_values.apply(lambda feature: y_pred - feature)
  abs_error_wo_feature = y_pred_wo_feature.apply(lambda feature: (y_true - feature).abs())
  error_contribution = abs_error_wo_feature.apply(lambda feature: abs_error - feature).mean().rename("error_contribution")

  return prediction_contribution, error_contribution

In [6]:
skf = StratifiedKFold(n_splits=5)

In [7]:
rfe_error_list = []

for i, (train_index, val_index) in enumerate(skf.split(X_train, LabelEncoder().fit_transform(y_train))):
    x_train_fold, x_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # prediction_contribution_train, error_contribution_train = get_feature_contributions(y_train_fold, y_train_pred, shap_values_train)
    # prediction_contribution_val, error_contribution_val = get_feature_contributions(y_val_fold, y_val_pred, shap_values_val)

    # contributions_train = pd.concat([prediction_contribution_train, error_contribution_train], axis=1)
    # contributions_val = pd.concat([prediction_contribution_val, error_contribution_val], axis=1)

    rfe_error = pd.DataFrame(dtype=float)
    features_curr = list(x_train_fold.columns.copy())
    feature_drop = None

    for iteration in range(len(x_train_fold.columns)):

        model = RandomForestRegressor(n_estimators=30)
        model.fit(x_train_fold[features_curr], y_train_fold)
        y_train_pred = model.predict(x_train_fold[features_curr])
        y_val_pred = model.predict(x_val_fold[features_curr])
        y_test_pred = model.predict(X_test[features_curr])

        shap_values_train = get_shap_values(model, x_train_fold[features_curr])
        shap_values_val = get_shap_values(model, x_val_fold[features_curr])
    
        prediction_contribution_val, error_contribution_val = get_feature_contributions(
            y_true=y_val_fold, 
            y_pred=y_val_pred, 
            shap_values=shap_values_val
        )

        rfe_error.loc[iteration, "feature_drop"] = feature_drop
        rfe_error.loc[iteration, "n_features"] = len(features_curr)
        rfe_error.loc[iteration, "y_pred_test"] = str(list(y_test_pred))
        rfe_error.loc[iteration, "contrib"] = error_contribution_val.max()
        rfe_error.loc[iteration, "mae_trn"] = mean_absolute_error(y_train_fold, y_train_pred)
        rfe_error.loc[iteration, "mae_val"] = mean_absolute_error(y_val_fold, y_val_pred)
        rfe_error.loc[iteration, "mae_tst"] = mean_absolute_error(y_test, y_test_pred)
        rfe_error.loc[iteration, "rmse_trn"] = mean_squared_error(y_train_fold, y_train_pred, squared=False)
        rfe_error.loc[iteration, "rmse_val"] = mean_squared_error(y_val_fold, y_val_pred, squared=False)
        rfe_error.loc[iteration, "rmse_tst"] = mean_squared_error(y_test, y_test_pred, squared=False)
        
        feature_drop = error_contribution_val.idxmax()
        features_curr.remove(feature_drop)
    
    rfe_error_list.append(rfe_error)
    rfe_error.to_csv(f'./output/rfe_error_{i}_fold.csv', index=False)

## Check best models

In [33]:
for error_df in rfe_error_list:
    print(error_df.loc[error_df['rmse_val'].idxmin()]['rmse_tst'])

0.699330837352168
0.685635709516996
0.7054019037953726
0.7196152459130766
0.7031785871654715


## Combine outputs

In [38]:
preds = []
for error_df in rfe_error_list:
    preds.append(ast.literal_eval(error_df.loc[error_df['rmse_val'].idxmin()]['y_pred_test']))

In [39]:
print(mean_absolute_error(y_test, np.mean(preds, axis=0)))
print(mean_squared_error(y_test, np.mean(preds, axis=0), squared=False))

0.5182392473118281
0.6789689830384856
